Patterns based on all time between different levels of change. Best:

In [15]:
#Import functions
import polars as pl
from functools import partial
from itertools import product, combinations
import multiprocessing
from numba import jit
import numpy as np
import pickle

In [16]:
#Load dataframe from CSV
currentDf=pl.read_csv("../Data/SP500/minuteHist2021/tradingHours/AAPL.csv", infer_schema_length=None)

In [17]:
#Get prices and timestamps from dataframe
openPrices=currentDf['open'].to_numpy()
closePrices=currentDf['close'].to_numpy()
timestamps=currentDf['time'].to_numpy()

In [18]:
#Join data into one 2d array
dataArray = np.dstack((openPrices,closePrices, timestamps))[0]
len(dataArray)

259972

In [20]:
#Algo
#Numba for speed
@jit(nopython=True, nogil=True)
#Takes in list of paramaters and an array with all the data
def algo(paramsList, dataArray):
    fee=paramsList[0][0] #Percent change in price to look for
    timeout=int(paramsList[0][1]) #The timeout when it takes too long
    divisor=int(paramsList[0][2]) #Divisor for ema
    allParams=paramsList[1:]
    openPrices=dataArray[:,0]
    
    #params has an array of arrays with ticks before, change, version
    
    tradePercent=0
    totalReturn=1

    total=0

    start=60 #Start of data
    end=int(len(openPrices)-61) #End of data

    for j in range(start, end):

        #Check that its not within the first or last hour of trading
        if 0<dataArray[j][2]-dataArray[j-60][2]<4500000 and 0<dataArray[j+60][2]-dataArray[j][2]<4500000:
            
        #Check that its all within one day of trading
        #if 0<dataArray[j][2]-dataArray[j-(gap*len(params))][2]<(60000*gap*len(params)*1.25) and 0<dataArray[j+timeout][2]-dataArray[j][2]<(60000*timeout*1.25):

        #Check that its all within first or last hour of trading
        #if 4500000<dataArray[j][2]-dataArray[j-60][2] or 4500000<dataArray[j+60][2]-dataArray[j][2]:

            #Set start price and continue
            startPrice=dataArray[j][0]
            continueParams=True

            for param in allParams:
                ticksBefore = int(param[0])
                change = param[1]
                version = param[2]

                #Set price we are checking
                if version==1:
                    checkPrice=dataArray[j-ticksBefore][0]


                elif version==2:

                    checkTotal = 0
                    #Exponential average by using equation of (1/2 * 1/2^n * price) (a/1-r) (1/3 * 2/3^n * price) which = 1 as it tends to infinity
                    expoError=0
                    divisor=4

                    count=0
                    for k in range(j-ticksBefore+1,j+1):
                        checkTotal+=(int(dataArray[k][0]) * (1/divisor * (1-(1/divisor))**(ticksBefore-count)))
                        count+=1

                    count=0
                    for k in range(j-ticksBefore+1,j+1):
                        expoError+=(1/divisor * (1-(1/divisor))**(ticksBefore-count))
                        count+=1

                    checkPrice = checkTotal/expoError


                elif version==3:
                    #checkPrice=sma(ticksBefore, j, openPrices)
                    checkTotal = 0
                    #Simple mean by add and divide
                    for k in range(j-ticksBefore+1,j+1):
                        checkTotal+=int(dataArray[k][0])
                    checkPrice = checkTotal/ticksBefore


                #If price now is above price we are checking, then continue
                if not checkPrice*(change+0.0001)>startPrice>checkPrice*change:
                    continueParams = False

            if continueParams:

                #Add to tally
                total+=1

                #Return on investment = new price / old price

                returnValue = (dataArray[j+timeout][0]-fee)/dataArray[j][0]
                totalReturn*=returnValue
                tradePercent+=returnValue
        
    if total>0:
        tradePercent=tradePercent/total
        
    #Return the average trade percent, the total number of times, and the parameters
    returnList = [tradePercent, totalReturn, total] + [param for param in allParams for param in param]
    return returnList
    # print(tradePercent)
    # print(tradePerDay)

#algo with all the dataframes already passed through
partialAlgo = partial(algo, dataArray=dataArray)

In [22]:
#Test algo to compile it
algo(np.array([[0, 50.00, 0],[10, 1.0, 3]]), dataArray)

[1.0002206719273488, 1.1500154803647593, 663.0, 10.0, 1.0, 3.0]

In [23]:
#Params
#rough commission fee
fee=[0.005]

#change
changeList=np.arange(1.000,1.001,0.0001)

#timeout
timeoutList=[1,2,3,5,10,20,30,50]

#ema sma price before
emaTicksBeforeList=[20,50]
smaTicksBeforeList=[3,5,10,20,50]
#days before list
priceTicksBeforeList=[1,2,3,5,10,20,30,50]


priceParamProduct = list(product(priceTicksBeforeList, changeList, [1]))
#emaParamProduct = list(product(emaTicksBeforeList, changeList, [2]))
smaParamProduct = list(product(smaTicksBeforeList, changeList, [3]))
allParamProduct = priceParamProduct + smaParamProduct

allSettingsProduct = list(product(fee, timeoutList, fee))

depth = 2
xParamCombinations = list(combinations(allParamProduct, depth))

fullCombinations=[]
for a in allSettingsProduct:
    for b in xParamCombinations:
        fullCombinations.append(np.array(([a] + [c for c in b])))
len(fullCombinations)

67080

In [24]:
#Run the algo over the full combinations list with multiprocessing

#List of results to store in right order
allResults=[]

#Start multiprocessing
with multiprocessing.Pool(15) as pool:
    for result in pool.map(
        partialAlgo,
        fullCombinations
    ):
        #Check if the total tally is high enough to be reasonable
        allResults.append(result)

In [25]:
#Save results in a pickle file
file_path = "../Results/sp500/V1/AAPL-minute2021-midday-67kParams-incFees.pkl"

# Save the allResults variable using pickle
with open(file_path, "wb") as file:
    pickle.dump(allResults, file)

In [26]:
#Minimum number of trades to be considered
minTotal=10
maxTotal=100000
results=[]

for result in allResults:
    if maxTotal>=result[2]>=minTotal:
        results.append(result)

In [27]:
#Sort results by each trade and by total
usedResults = results

sortedResults = sorted(usedResults.copy(), reverse=True)
sortedTotalResults = sorted(usedResults.copy(), reverse=True, key=lambda x: x[1])

In [28]:
#Return, Total return, number of times, ticks before, change, version: 1-price 2-ema 3-sma
for k in range(20):
    print(sortedResults[k])

[1.0034580387587884, 1.038534459629362, 11.0, 5.0, 1.0007, 1.0, 50.0, 1.0, 3.0]
[1.0032156606191642, 1.0325789279083892, 10.0, 5.0, 1.0003, 3.0, 20.0, 1.0, 3.0]
[1.0030687087010635, 1.031030305629927, 10.0, 2.0, 1.0008, 1.0, 20.0, 1.0008, 3.0]
[1.003041434778289, 1.0307863006517661, 10.0, 10.0, 1.0, 3.0, 50.0, 1.0003, 3.0]
[1.0029396386765599, 1.0353673875639364, 12.0, 1.0, 1.0007, 1.0, 20.0, 1.0004, 3.0]
[1.0029126181494465, 1.0445527114393358, 15.0, 50.0, 1.0, 1.0, 10.0, 1.0003, 3.0]
[1.002795411077671, 1.0367989108492126, 13.0, 2.0, 1.0, 1.0, 20.0, 1.0, 3.0]
[1.0027211538846938, 1.0274684777754297, 10.0, 50.0, 1.0, 1.0, 20.0, 1.0003, 3.0]
[1.0027102795543932, 1.0328775980415537, 12.0, 50.0, 1.0, 1.0, 3.0, 1.0, 3.0]
[1.0026021375371514, 1.026197659859535, 10.0, 10.0, 1.0, 3.0, 50.0, 1.0004, 3.0]
[1.0025778970603576, 1.0286014726378172, 11.0, 2.0, 1.0, 1.0, 10.0, 1.0, 3.0]
[1.002560104831138, 1.0257924735546309, 10.0, 2.0, 1.0006, 1.0, 20.0, 1.0003, 3.0]
[1.002528622653669, 1.03069484

In [29]:
for k in range(20):
    print(sortedTotalResults[k])

[1.000285288961297, 1.4128873305156127, 1236.0, 1.0, 1.0, 1.0, 2.0, 1.0002, 1.0]
[1.0001493743781897, 1.363851678413631, 2178.0, 3.0, 1.0008, 3.0, 5.0, 1.0008, 3.0]
[1.0001896570683027, 1.2905525090914558, 1394.0, 3.0, 1.0006, 3.0, 10.0, 1.0006, 3.0]
[1.0002456826259138, 1.289312851088719, 1060.0, 2.0, 1.0002, 1.0, 3.0, 1.0002, 1.0]
[1.0003129438931595, 1.2753974658535272, 795.0, 10.0, 1.0004, 3.0, 20.0, 1.0004, 3.0]
[1.0001317183574707, 1.275211552542677, 1942.0, 3.0, 1.0006, 3.0, 5.0, 1.0006, 3.0]
[1.000413528783224, 1.2741150166452142, 593.0, 1.0, 1.0001, 1.0, 5.0, 1.0006, 1.0]
[1.0001733742815841, 1.2662956020268166, 1417.0, 5.0, 1.0006, 3.0, 10.0, 1.0006, 3.0]
[1.0002056074390628, 1.2538440060865155, 1131.0, 10.0, 1.0008, 3.0, 20.0, 1.0008, 3.0]
[1.0004495820111798, 1.251939387982007, 505.0, 3.0, 1.0002, 1.0, 5.0, 1.0, 1.0]
[1.0002122165869571, 1.2483995448386447, 1073.0, 3.0, 1.0008, 3.0, 20.0, 1.0008, 3.0]
[1.0005337547878792, 1.2481481573689692, 421.0, 1.0, 1.0001, 1.0, 2.0, 1.